# Measuring Circadian Effect on Individual Player Stats
The circadian rhythm is the body's natural process of regulating sleep. In sports, the effects of traveling across time zones on athletic performance is attributed to the circadian rhythm. The goal of this notebook is to evaluate if there is a significant effect of travel on individual player performance of Erie Otters' players.

In [1]:
import pandas as pd
import numpy as np
import pprint
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt

## Data Wrangling
We need to get stats for individual games for each player on the Erie Otters. Also, the location (home/away) of games needs to be tracked.

In [2]:
# Read in data set
path = 'data/df_all_events.pkl'
df = pd.read_pickle(path)

In [3]:
# convert dataframe to iterable python dict
events = df.to_dict(orient='records')

In [38]:
# count the number of events per game for each player
players = {}
cur_date = ''
game_dict = {
    'home' : 0,
    'dump_retained': 0,       # successful dump in attempt
    'dump_lost': 0,           # unsuccessful dump in attempt
    'faceoff_win': 0,         # face off won
    'faceoff_lost': 0,        # face off lost
    'goals': 0,               # goals
    'shots_attempts': 0,      # total shots
    'shots_on_net': 0,        # number of shots on goal
    'shots_missed': 0,        # number of attempted shots that miss net
    'shots_blocked': 0,       # number of attempted shots that are blocked
    'zone_entry_carried': 0,  # puck carried into offensive zone
    'zone_entry_dumped': 0,   # puck dumped into offesnive zone
    'zone_entry_played': 0,   # puck played into offesnive zone via pass
    'penalty_taken': 0,       # number of penalties taken
    'play_indirect': 0,       # successful indirect passes (i.e. pass rimmed of boards)
    'play_direct': 0,         # successful direct passes (i.e. tape to tape pass)
    'inc_play_indirect': 0,   # unsuccessful indirect passes (i.e. pass rimmed of boards)
    'inc_play_direct': 0,     # unsuccessful direct passes (i.e. tape to tape pass)
}

date_change = 0
for e in events:
    if e['game_date'] != cur_date:
        date_change += 1
        cur_date = e['game_date']
        
    # identify faceoffs lost
    event_name = ''
    if e['team'] != 'Erie Otters' and e['event'] == 'Faceoff Win':
        player_name = e['player_2']
        event_name = 'faceoff_lost'
    elif e['team'] != 'Erie Otters':  # no other events where team name is not Erie need to be tracked
        continue
    else:
        player_name = e['player_1']
    
    # identify event that happend
    if e['event'] == 'Dump In/Out' and e['detail_1'] == 'Lost': event_name = 'dump_lost'
    elif e['event'] == 'Dump In/Out' and e['detail_1'] == 'Retained': event_name = 'dump_retained'
    elif e['event'] == 'Faceoff Win' and len(event_name) < 1: event_name = 'faceoff_win'
    elif e['event'] == 'Goal': event_name = 'goals'   
    elif e['event'] == 'Shot': 
        if e['detail_2'] == 'On Net': event_name = 'shots_on_net'
        elif e['detail_2'] == 'Missed': event_name = 'shots_missed'
        elif e['detail_2'] == 'Blocked': event_name = 'shots_blocked'
    elif e['event'] == 'Zone Entry' and e['detail_1'] == 'Carried': event_name = 'zone_entry_carried'
    elif e['event'] == 'Zone Entry' and e['detail_1'] == 'Dumped': event_name = 'zone_entry_dumped'
    elif e['event'] == 'Zone Entry' and e['detail_1'] == 'Played': event_name = 'zone_entry_played'
    elif e['event'] == 'Penalty Taken': event_name = 'penalty_taken'
    elif e['event'] == 'Play' and e['detail_1'] == 'Indirect': event_name = 'play_indirect'
    elif e['event'] == 'Play' and e['detail_1'] == 'Direct': event_name = 'play_direct'
    elif e['event'] == 'Incomplete Play' and e['detail_1'] == 'Indirect': event_name = 'inc_play_indirect'
    elif e['event'] == 'Incomplete Play' and e['detail_1'] == 'Direct': event_name = 'inc_play_direct'

    # check if this player's name is in the players dict, if not, add them
    if player_name not in players:
        players[player_name] = {}
        
    # check if this game's date is in this player's stat dict, if not, copy a 'game_dict' to this player's dict
    if cur_date not in players[player_name]:
        players[player_name][cur_date] = game_dict.copy()
        if e['home_team'] == 'Erie Otters':
            players[player_name][cur_date]['home'] = 1
    
    # if identified an event we care about, increment that stat
    if len(event_name) > 0:
        players[player_name][cur_date][event_name] += 1

# count number of home/away games
count_home = 0
count_away = 0
for player, game_date in players.items():
    for game, game_stats in game_date.items():
        if game_stats['home'] == 1: count_home +=1
        else: count_away += 1
print('num home games for all players: ' + str(count_home))
print('num away games for all players: ' + str(count_away))

num home games for all players: 363
num away games for all players: 395


In [57]:
# using the above data wrangling step, build a dict that stores player averages for home and away games
## this will help to identify possible players that demonstrate increased/decreased adaptability 
## to away game environments
game_dict = {
    'home': 0,
    'dump_retained': 0,       # successful dump in attempt
    'dump_lost': 0,           # unsuccessful dump in attempt
    'faceoff_win': 0,         # face off won
    'faceoff_lost': 0,        # face off lost
    'goals': 0,               # goals
    'shots_attempts': 0,      # total shots
    'shots_on_net': 0,        # number of shots on goal
    'shots_missed': 0,        # number of attempted shots that miss net
    'shots_blocked': 0,       # number of attempted shots that are blocked
    'zone_entry_carried': 0,  # puck carried into offensive zone
    'zone_entry_dumped': 0,   # puck dumped into offesnive zone
    'zone_entry_played': 0,   # puck played into offesnive zone via pass
    'penalty_taken': 0,       # number of penalties taken
    'play_indirect': 0,       # successful indirect passes (i.e. pass rimmed of boards)
    'play_direct': 0,         # successful direct passes (i.e. tape to tape pass)
    'inc_play_indirect': 0,   # unsuccessful indirect passes (i.e. pass rimmed of boards)
    'inc_play_direct': 0,     # unsuccessful direct passes (i.e. tape to tape pass)
}

players_avg = {}
players_avg_home = {}
players_avg_away = {}

for player, games in players.items():
    players_avg[player] = {}
    players_avg[player]['home'] = game_dict.copy()
    players_avg[player]['away'] = game_dict.copy()
    num_home_games = 0
    num_away_games = 0

    # append all stats, count home and away games
    for date, game in games.items():
        if game['home'] == 1:
            num_home_games += 1
            for stat, value in game.items():
                players_avg[player]['home'][stat] += value
        elif game['home'] == 0:
            num_away_games += 1
            for stat, value in game.items():
                players_avg[player]['away'][stat] += value
    
#     print(player + '\'s home games: ' + str(num_home_games))
#     print(player + '\'s away games: ' + str(num_away_games))
#     print()
    
    # compute averages of stat
    for stat, value in players_avg[player]['home'].items():
        players_avg[player]['home'][stat] = players_avg[player]['home'][stat] / num_home_games
    for stat, value in players_avg[player]['away'].items():
        players_avg[player]['away'][stat] /= num_away_games

    # keep separate dicts for home and away averages (this makes data analysis step easier)
    players_avg_home[player] = players_avg[player]['home'].copy()
    players_avg_away[player] = players_avg[player]['away'].copy()

In [56]:
pprint.pprint(players_avg_home)

{'Aidan Campbell': {'dump_lost': 0.125,
                    'dump_retained': 0.0,
                    'faceoff_lost': 0.0,
                    'faceoff_win': 0.0,
                    'goals': 0.0,
                    'home': 1.0,
                    'inc_play_direct': 0.75,
                    'inc_play_indirect': 1.25,
                    'penalty_taken': 0.0,
                    'play_direct': 2.0,
                    'play_indirect': 8.875,
                    'shots_attempts': 0.0,
                    'shots_blocked': 0.0,
                    'shots_missed': 0.0,
                    'shots_on_net': 0.0,
                    'zone_entry_carried': 0.0,
                    'zone_entry_dumped': 0.0,
                    'zone_entry_played': 0.0},
 'Alex Gritz': {'dump_lost': 0.0,
                'dump_retained': 0.0,
                'faceoff_lost': 3.0,
                'faceoff_win': 4.0,
                'goals': 0.0,
                'home': 1.0,
                'inc_play_direct': 3.0,
 

## Data Analysis


In [67]:
df_player_avg_home = pd.DataFrame.from_dict(players_avg_home, orient='index')
df_player_avg_home['shots_attempts'] = df_player_avg_home['shots_blocked'] + df_player_avg_home['shots_on_net'] + df_player_avg_home['shots_missed'] 

df_player_avg_away = pd.DataFrame.from_dict(players_avg_away, orient='index')
df_player_avg_away['shots_attempts'] = df_player_avg_away['shots_blocked'] + df_player_avg_away['shots_on_net'] + df_player_avg_away['shots_missed'] 

df_player_avg_diff = df_player_avg_home - df_player_avg_away

In [68]:
df_player_avg_diff

,home,dump_retained,dump_lost,faceoff_win,faceoff_lost,goals,shots_attempts,shots_on_net,shots_missed,shots_blocked,zone_entry_carried,zone_entry_dumped,zone_entry_played,penalty_taken,play_indirect,play_direct,inc_play_indirect,inc_play_direct
Connor Lockhart,1.0,0.111111,-0.122222,0.677778,1.638889,-0.038889,1.416667,0.672222,0.705556,0.038889,0.472222,0.533333,0.027778,0.022222,0.466667,2.122222,-0.483333,-0.405556
Hayden Fowler,1.0,0.352941,-0.352941,-0.529412,-2.529412,0.117647,-0.294118,-0.176471,-0.235294,0.117647,-0.176471,-0.117647,-0.235294,-0.705882,0.764706,-0.058824,-0.411765,0.941176
Jamie Drysdale,1.0,0.066667,-0.866667,0.000000,0.000000,-0.266667,1.000000,0.733333,0.133333,0.133333,0.000000,0.000000,0.466667,0.000000,0.133333,2.266667,0.200000,1.466667
Kurtis Henry,1.0,-0.025063,-0.989975,0.000000,0.000000,-0.090226,1.328321,0.511278,0.363409,0.453634,0.160401,-0.155388,0.110276,-0.456140,0.370927,-0.614035,-0.007519,-0.062657
Brendan Sellan,1.0,0.423559,-0.473684,-0.042607,-0.185464,0.072682,0.313283,-0.100251,0.343358,0.070175,-0.300752,0.636591,-0.040100,0.115288,0.566416,-0.842105,0.293233,0.596491
Danial Singer,1.0,-0.355556,-0.011111,-0.011111,-0.033333,-0.566667,-0.066667,-0.077778,-0.022222,0.033333,-0.166667,0.077778,0.600000,0.200000,0.211111,1.600000,-1.066667,-0.022222
Austen Swankler,1.0,-0.070175,0.172515,0.192982,0.619883,0.070175,0.371345,0.564327,-0.137427,-0.055556,1.324561,0.236842,0.190058,-0.239766,1.982456,3.432749,0.131579,0.114035
Jacob Golden,1.0,-0.294118,0.294118,0.000000,0.000000,-0.058824,0.882353,0.235294,0.058824,0.588235,0.529412,0.176471,-0.176471,-0.176471,0.235294,2.294118,0.941176,0.823529
Emmett Sproule,1.0,-0.002506,-0.085213,-0.032581,-0.122807,-0.080201,0.446115,0.451128,-0.100251,0.095238,-0.190476,0.092732,0.087719,0.010025,0.578947,0.203008,0.674185,-0.385965
Jack Duff,1.0,0.203008,-0.604010,0.000000,0.000000,0.105263,0.248120,0.140351,-0.070175,0.177945,-0.310777,-0.150376,0.225564,0.012531,-0.481203,1.456140,-0.360902,-0.734336
